In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list): # = "sorted" function
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data


def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data


def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo

def constantTF(listo):
    first=listo[0]
    ln=len(listo)
    for j in [0..ln-2]:
        if listo[j]!=listo[j+1]:return False
    return True

def push(listo):
    pushed=[]
    lenth=len(listo)
    pushed=pushed+[listo[lenth-1]]
    for j in [0..lenth-2]:
        pushed=pushed+[listo[j]]
    return pushed

def kIteratesofPush(listo,k):
    if k == 0:return listo
    answer=listo
    for k in [1..k]:
        answer=push(answer)
    return answer

def cycleEquivalentTF(list1,list2):
    len1=len(list1)
    len2=len(list2)
    if len1 != len2:return False
    if list1==list2:return True
    test=list2
    for k in [1..len2]:
        test=push(test)
        if test==list1:return True
    return False

def fromDigitsToNumber(listo,prime):
    ln=len(listo)
    sum=0
    for j in [0..ln-1]:
        digit=listo[j]
        sum=sum+digit*prime^j
    return sum

def numberListToExpansionList(seq,prime):
    el=[]
    ln=len(seq)
    for j in [0..ln-1]:
        nbr=seq[j]
        dgt=nbr.digits(prime)
        el=el+[dgt]
    return el

def expansionListToNumberList(listlist,prime):
    nlist=[]
    ln=len(listlist)
    if ln==0:return []
    for j in [0..ln-1]:
        lst=listlist[j]
        nbr=fromDigitsToNumber(lst,prime)
        nlist=nlist+[nbr]
    return nlist
        
def f5(seq, idfun=None): # https://www.peterbe.com/plog/uniqifiers-benchmark
    if idfun is None:
        def idfun(x): return x
        seen = {}
        result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result

def drop_duplicates(seq):return f5(seq,idfun=None) # for numerical sequences only

def dropCycle_EquivalentLists(listlist): # drops duplicates
    if listlist==[]:return []
    ddl=[listlist[0]]
    for lst in listlist:
        flag = 1
        for listo in ddl:
            if cycleEquivalentTF(lst,listo):
                flag=0
        if flag==1:ddl=ddl+[lst]
    return ddl  

def zeroLogsOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def zeroLogsOverFieldDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            di=i.digits(prime)
            answer=answer+[di]
    return answer

def last(listo):
    if listo==[]:return None
    if len(listo)>0:return listo[len(listo)-1]

def cycleEquivalenceClass(listo):
    if listo==[]:return []
    classs=[listo]
    while len(classs)<len(listo):
        classs=classs+[push(last(classs))]
    return classs
    

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [8]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
no=[]
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    residue=mod(index,prime)
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    lm=logsAndMultiplicitiesOverField(poly,prime,sd)
    logs=[]
    digits=[]
    for j in [0..len(lm)-1]:
        log=lm[j][0]
        logs=logs+[log]
    logs=sorted(logs)
    count=0
    for j in [0..len(lm)-1]:
        log=logs[j]
        digit=log.digits(prime)
        digit=padListOnRightToLengthK(digit,sd)
        tf=constantTF(digit)
        if tf==False:count=count+1
    print (index,residue,sd,count,count/sd)

(-1, 6, 1, 0, 0)
(0, 0, 1, 0, 0)
(1, 1, 4, 4, 1)
(2, 2, 2, 2, 1)
(3, 3, 2, 6, 3)
(4, 4, 6, 8, 4/3)
(5, 5, 5, 10, 2)
(6, 6, 1, 0, 0)
(7, 0, 4, 6, 3/2)
(8, 1, 4, 4, 1)
(9, 2, 2, 2, 1)
(10, 3, 2, 6, 3)
(11, 4, 6, 8, 4/3)
(12, 5, 5, 10, 2)
(13, 6, 1, 0, 0)
(14, 0, 4, 6, 3/2)
(15, 1, 4, 4, 1)
(16, 2, 2, 2, 1)
(17, 3, 2, 6, 3)
(18, 4, 6, 8, 4/3)
(19, 5, 5, 10, 2)
(20, 6, 1, 0, 0)
(21, 0, 4, 6, 3/2)
(22, 1, 4, 4, 1)
(23, 2, 2, 2, 1)
(24, 3, 2, 6, 3)


KeyboardInterrupt: 

In [15]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
residue=0
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(0, 2)
logs:
[3, 15]
cycle-equivalence class representatives as expansions
[[3]]
cycle-equivalence class representatives as numbers
[3]
------------------------------------------------------------------------------
(5, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(10, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(15, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(20, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers


In [16]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
residue=1
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(1, 4)
logs:
[91, 143, 403, 455]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(6, 4)
logs:
[91, 143, 156, 312, 403, 455, 468, 624]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(11, 4)
logs:
[91, 143, 156, 312, 403, 455, 468, 624]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(16, 4)
logs:
[91, 143, 156, 312, 403, 455, 468, 624]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(2

cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(176, 4)
logs:
[91, 143, 156, 312, 403, 455, 468, 624]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(181, 4)
logs:
[91, 143, 156, 312, 403, 455, 468, 624]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(186, 4)
logs:
[91, 143, 156, 312, 403, 455, 468, 624]
cycle-equivalence class representatives as expansions
[[1, 3, 3]]
cycle-equivalence class representatives as numbers
[91]
------------------------------------------------------------------------------
(191, 4)
logs:
[91, 143, 156, 3

In [17]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
residue=2
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(2, 4)
logs:
[156, 169, 221, 468, 481, 533]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
------------------------------------------------------------------------------
(7, 4)
logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
------------------------------------------------------------------------------
(12, 4)
logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
------------------------------------------------------------------------------
(17, 4)
logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
---------------------------------------------------

logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
------------------------------------------------------------------------------
(172, 4)
logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
------------------------------------------------------------------------------
(177, 4)
logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
------------------------------------------------------------------------------
(182, 4)
logs:
[156, 169, 221, 312, 468, 481, 533, 624]
cycle-equivalence class representatives as expansions
[[4, 3, 1, 1]]
cycle-equivalence class representatives as numbers
[169]
--------------------------------------------

In [18]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
residue=3
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(3, 6)
logs:
[2961, 3717, 3906, 6993, 10773, 11529, 11718, 14805]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 4]]
cycle-equivalence class representatives as numbers
[2961]
------------------------------------------------------------------------------
(8, 6)
logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 4]]
cycle-equivalence class representatives as numbers
[2961]
------------------------------------------------------------------------------
(13, 6)
logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 4]]
cycle-equivalence class representatives as numbers
[2961]
------------------------------------------------------------------------------
(18, 6)
logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 

logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 4]]
cycle-equivalence class representatives as numbers
[2961]
------------------------------------------------------------------------------
(153, 6)
logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 4]]
cycle-equivalence class representatives as numbers
[2961]
------------------------------------------------------------------------------
(158, 6)
logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1, 2, 3, 3, 4]]
cycle-equivalence class representatives as numbers
[2961]
------------------------------------------------------------------------------
(163, 6)
logs:
[2961, 3717, 3906, 6993, 7812, 10773, 11529, 11718, 14805, 15624]
cycle-equivalence class representatives as expansions
[[1,

In [19]:
# segregate indices by residue class & filter out logs w. constant expansions
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
residue=4
def f(listo):return fromDigitsToNumber(listo,prime)
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        print (index,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        digits=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        logs=sorted(logs)
        digits=[]
        for j in [0..len(lm)-1]:
            log=logs[j]
            digit=log.digits(prime)
            digit=padListOnRightToLengthK(digit,sd)
            digits=digits+[digit]
        print "logs:"
        print logs
        #print "digits:";print digits
        classes=[]
        for j in [0..len(digits)-1]: #filter out expansions of form  (t,t,...,t)
            dgt=digits[j]
            if constantTF(dgt)==False:
                mc=cycleEquivalenceClass(dgt)
                classes=classes+[mc[0]] # mc[0] is a representative of the equivalence class
        dropped=dropCycle_EquivalentLists(classes)
        #print "class representatives for non-constant expansions:"
        #print dropped
        numbers=sorted(expansionListToNumberList(dropped,prime))
        expansions=numberListToExpansionList(numbers,prime)
        print "cycle-equivalence class representatives as expansions"
        print expansions
        print "cycle-equivalence class representatives as numbers"
        print numbers
        print "------------------------------------------------------------------------------"

(-1, 1)
logs:
[]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(4, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(9, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(14, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(19, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------

(189, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(194, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
(199, 1)
logs:
[1, 2, 3, 4]
cycle-equivalence class representatives as expansions
[]
cycle-equivalence class representatives as numbers
[]
------------------------------------------------------------------------------
